In [1]:
# import resnet 18
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import io
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.model_selection import train_test_split
import numpy as np
import os
import csv
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from time import perf_counter


In [ ]:

# Read and transform images
def read_image(image_path):
    img = io.read_image(image_path)
    weights = ResNet18_Weights.DEFAULT
    transform = weights.transforms()
    return transform(img)

# Training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss, correct = 0, 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()

    train_accuracy = correct / len(train_loader.dataset)
    train_loss /= len(train_loader)
    return train_loss, train_accuracy

# Testing function
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()

    test_accuracy = correct / len(test_loader.dataset)
    test_loss /= len(test_loader)
    return test_loss, test_accuracy

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.continent_mapping = {continent: idx for idx, continent in enumerate(self.data['continent'].unique())}
        self.data['continent_label'] = self.data['continent'].map(self.continent_mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx]['image_name'])
        image = io.read_image(img_path)
        if self.transform:
            image = self.transform(image)
        label = self.data.iloc[idx]['continent_label']
        return image, label

# Main training loop
def train_loop(csv_path, root_dir, num_epochs=15, batch_size=32, learning_rate=0.001, weight_decay=0.0001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Data transformations
    weights = ResNet18_Weights.DEFAULT
    transform = weights.transforms()

    # Dataset and DataLoaders
    dataset = CustomDataset(csv_file=csv_path, root_dir=root_dir, transform=transform)
    train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
    train_set = torch.utils.data.Subset(dataset, train_idx)
    test_set = torch.utils.data.Subset(dataset, test_idx)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    # Load ResNet18 and modify the final layer
    resnet = resnet18(weights=ResNet18_Weights.DEFAULT)
    for param in resnet.parameters():
        param.requires_grad = False  # Freeze all layers except the last
    num_features = resnet.fc.in_features
    resnet.fc = nn.Linear(num_features, 6)  # 6 continents
    resnet.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet.fc.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Training and evaluation
    metrics = pd.DataFrame(columns=['epoch', 'train_loss', 'train_accuracy', 'test_loss', 'test_accuracy'])

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train(resnet, train_loader, optimizer, criterion, device)
        test_loss, test_accuracy = test(resnet, test_loader, criterion, device)

        # Save metrics and model
        metrics = metrics.append({'epoch': epoch, 'train_loss': train_loss, 'train_accuracy': train_accuracy,
                                  'test_loss': test_loss, 'test_accuracy': test_accuracy}, ignore_index=True)
        metrics.to_csv('metrics.csv', index=False)
        torch.save(resnet.state_dict(), 'model.pth')

        print(f"Epoch {epoch + 1}/{num_epochs}: "
              f"Train Loss={train_loss:.4f}, Train Acc={train_accuracy:.4f}, "
              f"Test Loss={test_loss:.4f}, Test Acc={test_accuracy:.4f}")

    print("Training complete!")

# Example usage
# Ensure 'coords_processed.csv' and 'dataset/' are properly set up
train_loop(csv_path='coords_processed.csv', root_dir='dataset/', num_epochs=15)


Using device: cpu


In [19]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124



Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/cu124/torch-2.5.1%2Bcu124-cp311-cp311-win_amd64.whl (2510.8 MB)
     ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
     --- ------------------------------------ 0.6/6.1 MB 18.5 MB/s eta 0:00:01
     ------- -------------------------------- 1.1/6.1 MB 14.2 MB/s eta 0:00:01
     ---------- ----------------------------- 1.6/6.1 MB 13.1 MB/s eta 0:00:01
     -------------- ------------------------- 2.2/6.1 MB 12.7 MB/s eta 0:00:01
     ----------------- ---------------------- 2.7/6.1 MB 12.4 MB/s eta 0:00:01
     --------------------- ------------------ 3.3/6.1 MB 12.4 MB/s eta 0:00:01
     ------------------------- -------------- 3.9/6.1 MB 12.3 MB/s eta 0:00:01
     ---------------------------- ----------- 4.4/6.1 MB 12.2 MB/s eta 0:00:01
     -------------------------------- ------- 4.9/6.1 MB 12.2 MB/s eta 0:00:01
     ----------------------------------- ---- 5

In [10]:
!nvidia-smi

Thu Jan  9 16:46:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   50C    P0             25W /  280W |    1230MiB /   8192MiB |     42%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
import torch

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("Current CUDA Version:", torch.version.cuda)
else:
    print("CUDA is not available. Check your installation.")


PyTorch Version: 2.5.1+cpu
CUDA Available: False
CUDA is not available. Check your installation.


In [15]:
torch.__version__

'2.5.1+cpu'